In [ ]:
!pip install tokenizers==0.13.2
# !pip install nltk==3.8.1


In [ ]:
!pip install nltk
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

In [ ]:
!file articles1.csv

articles1.csv: CSV text


In [ ]:

import pandas as pd

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import FreqDist
from nltk.util import bigrams
import string
import csv

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')
# Load the dataset
try:
  df = pd.read_csv("articles1.csv", error_bad_lines=False, encoding="latin-1")
  print(df)
except pd.errors.ParserError as e:
    print(f"Error reading CSV: {e}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
<ipython-input-15-790c60a7b6b2>:16: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv("articles1.csv", error_bad_lines=False, encoding="latin-1")
<ipython-input-15-790c60a7b6b2>:16: DtypeWarning: Columns (0,1,6,7,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,8

      Unnamed: 0     id                                              title  \
0              0  17283  House Republicans Fret About Winning Their Hea...   
1              1  17284  Rift Between Officers and Residents as Killing...   
2              2  17285  Tyrus Wong, âBambiâ Artist Thwarted by Rac...   
3              3  17286  Among Deaths in 2016, a Heavy Toll in Pop Musi...   
4              4  17287  Kim Jong-un Says North Korea Is Preparing to T...   
...          ...    ...                                                ...   
50114      53287  73465  Rex Tillerson Says Climate Change Is Real, but...   
50115      53288  73466  The Biggest Intelligence Questions Raised by t...   
50116      53289  73467  Trump Announces Plan That Does Little to Resol...   
50117      53290  73468    Dozens of For-Profit Colleges Could Soon Close    
50118      53291  73469                     The Milky Wayâs Stolen Stars   

          publication                         author        dat

In [ ]:
# df = df.drop('url',axis=1)
df.head()

,Unnamed: 0,id,title,publication,author,date,year,month,content,Unnamed: 9,...,Unnamed: 844,Unnamed: 845,Unnamed: 846,Unnamed: 847,Unnamed: 848,Unnamed: 849,Unnamed: 850,Unnamed: 851,Unnamed: 852,Unnamed: 853
0,0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,31-12-2016,2016,12,WASHINGTON â Congressional Republicans ha...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,17284,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,19-06-2017,2017,6,"After the bullet shells get counted, the blood...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,17285,"Tyrus Wong, âBambiâ Artist Thwarted by Rac...",New York Times,Margalit Fox,06-01-2017,2017,1,When Walt Disneyâs âBambiâ opened in 194...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,17286,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,10-04-2017,2017,4,"Death may be the great equalizer, but it isnâ...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,17287,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,02-01-2017,2017,1,"SEOUL, South Korea â North Koreaâs lead...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:

# Tokenizer function
def word_tokenizer(text):
    try:
        if pd.isna(text):
            return []  # Return an empty list for NaN values
        tokens = word_tokenize(str(text))
        tokens = [word.lower() for word in tokens if word.isalnum()]  # Remove punctuation
        return tokens
    except Exception as e:
        print(f"Error tokenizing: {e}")
        return []

# Tokenize the entire corpus
# corpus_tokens = [word_tokenizer(text) for text in df['content']]
corpus_tokens = [word_tokenizer(text) for text in df['content'].fillna('')]

# 1. The first 10 lines of the tokenizer's output for the whole corpus
for i, tokens in enumerate(corpus_tokens[:10]):
    print(f"Line {i + 1}: {tokens}\n")

# 2. The total number of tokens and types (unique tokens) in the corpus
all_tokens = [token for tokens in corpus_tokens for token in tokens]
total_tokens = len(all_tokens)
unique_tokens = len(set(all_tokens))

# 3. The type/token ratio for the corpus
type_token_ratio = unique_tokens / total_tokens

# 4. A list of the top 3 most frequent tokens, along with their frequencies
fdist_tokens = FreqDist(all_tokens)
top_3_tokens = fdist_tokens.most_common(3)

# 5. The number of tokens that appeared only once in the corpus
single_occurrence_tokens = len([token for token, freq in fdist_tokens.items() if freq == 1])

# 6. A list of the top 3 most frequent words (excluding punctuation and other symbols)
filtered_tokens = [token for token in all_tokens if token.isalpha()]
fdist_words = FreqDist(filtered_tokens)
top_3_words = fdist_words.most_common(3)

# 7. The lexical diversity (type/token ratio) when using only words
lexical_diversity_words = len(set(filtered_tokens)) / len(filtered_tokens)

# 8. A list of the top 3 most frequent words (excluding stopwords and punctuation)
stop_words = set(stopwo!pip install tokenizers==0.13.2
# !pip install nltk==3.8.1


!pip install nltk
import nltk
nltk.download('all')

import pandas as pd

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import FreqDist
from nltk.util import bigrams
import string
import csv

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')
# Load the dataset
try:
  df = pd.read_csv("articles1.csv", error_bad_lines=False, encoding="latin-1")
  print(df)
except pd.errors.ParserError as e:
    print(f"Error reading CSV: {e}")


# df = df.drop('url',axis=1)
df.head()


# Tokenizer function
def word_tokenizer(text):
    try:
        if pd.isna(text):
            return []  # Return an empty list for NaN values
        tokens = word_tokenize(str(text))
        tokens = [word.lower() for word in tokens if word.isalnum()]  # Remove punctuation
        return tokens
    except Exception as e:
        print(f"Error tokenizing: {e}")
        return []

# Tokenize the entire corpus
# corpus_tokens = [word_tokenizer(text) for text in df['content']]
corpus_tokens = [word_tokenizer(text) for text in df['content'].fillna('')]

# 1. The first 10 lines of the tokenizer's output for the whole corpus
for i, tokens in enumerate(corpus_tokens[:10]):
    print(f"Line {i + 1}: {tokens}\n")

# 2. The total number of tokens and types (unique tokens) in the corpus
all_tokens = [token for tokens in corpus_tokens for token in tokens]
total_tokens = len(all_tokens)
unique_tokens = len(set(all_tokens))

# 3. The type/token ratio for the corpus
type_token_ratio = unique_tokens / total_tokens

# 4. A list of the top 3 most frequent tokens, along with their frequencies
fdist_tokens = FreqDist(all_tokens)
top_3_tokens = fdist_tokens.most_common(3)

# 5. The number of tokens that appeared only once in the corpus
single_occurrence_tokens = len([token for token, freq in fdist_tokens.items() if freq == 1])

# 6. A list of the top 3 most frequent words (excluding punctuation and other symbols)
filtered_tokens = [token for token in all_tokens if token.isalpha()]
fdist_words = FreqDist(filtered_tokens)
top_3_words = fdist_words.most_common(3)

# 7. The lexical diversity (type/token ratio) when using only words
lexical_diversity_words = len(set(filtered_tokens)) / len(filtered_tokens)

# 8. A list of the top 3 most frequent words (excluding stopwords and punctuation)
stop_words = set(stopwords.words('english'))
filtered_tokens_no_stopwords = [token for token in filtered_tokens if token not in stop_words]
fdist_no_stopwords = FreqDist(filtered_tokens_no_stopwords)
top_3_words_no_stopwords = fdist_no_stopwords.most_common(3)

# 9. The lexical density (type/token ratio when using only word tokens without stopwords)
lexical_density_no_stopwords = len(set(filtered_tokens_no_stopwords)) / len(filtered_tokens_no_stopwords)

# 10. A list of the most frequent 3 bigrams (excluding stopwords and punctuation)
all_bigrams = list(bigrams(filtered_tokens_no_stopwords))
fdist_bigrams = FreqDist(all_bigrams)
top_3_bigrams = fdist_bigrams.most_common(3)

# results
print(f"2. Total Tokens: {total_tokens}, Unique Tokens: {unique_tokens}")
print(f"3. Type/Token Ratio: {type_token_ratio}")
print(f"4. Top 3 Tokens: {top_3_tokens}")
print(f"5. Single Occurrence Tokens: {single_occurrence_tokens}")
print(f"6. Top 3 Words (excluding punctuation): {top_3_words}")
print(f"7. Lexical Diversity (using only words): {lexical_diversity_words}")
print(f"8. Top 3 Words (excluding stopwords and punctuation): {top_3_words_no_stopwords}")
print(f"9. Lexical Density (using only word tokens without stopwords): {lexical_density_no_stopwords}")
print(f"10. Top 3 Bigrams (excluding stopwords and punctuation): {top_3_bigrams}")
rds.words('english'))
filtered_tokens_no_stopwords = [token for token in filtered_tokens if token not in stop_words]
fdist_no_stopwords = FreqDist(filtered_tokens_no_stopwords)
top_3_words_no_stopwords = fdist_no_stopwords.most_common(3)

# 9. The lexical density (type/token ratio when using only word tokens without stopwords)
lexical_density_no_stopwords = len(set(filtered_tokens_no_stopwords)) / len(filtered_tokens_no_stopwords)

# 10. A list of the most frequent 3 bigrams (excluding stopwords and punctuation)
all_bigrams = list(bigrams(filtered_tokens_no_stopwords))
fdist_bigrams = FreqDist(all_bigrams)
top_3_bigrams = fdist_bigrams.most_common(3)

# Display the results
print(f"2. Total Tokens: {total_tokens}, Unique Tokens: {unique_tokens}")
print(f"3. Type/Token Ratio: {type_token_ratio}")
print(f"4. Top 3 Tokens: {top_3_tokens}")
print(f"5. Single Occurrence Tokens: {single_occurrence_tokens}")
print(f"6. Top 3 Words (excluding punctuation): {top_3_words}")
print(f"7. Lexical Diversity (using only words): {lexical_diversity_words}")
print(f"8. Top 3 Words (excluding stopwords and punctuation): {top_3_words_no_stopwords}")
print(f"9. Lexical Density (using only word tokens without stopwords): {lexical_density_no_stopwords}")
print(f"10. Top 3 Bigrams (excluding stopwords and punctuation): {top_3_bigrams}")


Line 1: ['washington', 'congressional', 'republicans', 'have', 'a', 'new', 'fear', 'when', 'it', 'comes', 'to', 'their', 'health', 'care', 'lawsuit', 'against', 'the', 'obama', 'administration', 'they', 'might', 'win', 'the', 'incoming', 'trump', 'administration', 'could', 'choose', 'to', 'no', 'longer', 'defend', 'the', 'executive', 'branch', 'against', 'the', 'suit', 'which', 'challenges', 'the', 'authority', 'to', 'spend', 'billions', 'of', 'dollars', 'on', 'health', 'insurance', 'subsidies', 'for', 'and', 'americans', 'handing', 'house', 'republicans', 'a', 'big', 'victory', 'on', 'issues', 'but', 'a', 'sudden', 'loss', 'of', 'the', 'disputed', 'subsidies', 'could', 'conceivably', 'cause', 'the', 'health', 'care', 'program', 'to', 'implode', 'leaving', 'millions', 'of', 'people', 'without', 'access', 'to', 'health', 'insurance', 'before', 'republicans', 'have', 'prepared', 'a', 'replacement', 'that', 'could', 'lead', 'to', 'chaos', 'in', 'the', 'insurance', 'market', 'and', 'spur',

In [ ]:
import pandas as pd
import csv
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix
import re

# Load the dataset
data = pd.read_csv("Reviews.csv",quoting = csv.QUOTE_NONE, error_bad_lines=False)


Streaming output truncated to the last 5000 lines.
Skipping line 476232: expected 20 fields, saw 32
Skipping line 476249: expected 20 fields, saw 24
Skipping line 476262: expected 20 fields, saw 27
Skipping line 476288: expected 20 fields, saw 22
Skipping line 476316: expected 20 fields, saw 66
Skipping line 476317: expected 20 fields, saw 24
Skipping line 476337: expected 20 fields, saw 28
Skipping line 476343: expected 20 fields, saw 24
Skipping line 476362: expected 20 fields, saw 21
Skipping line 476363: expected 20 fields, saw 21
Skipping line 476380: expected 20 fields, saw 25
Skipping line 476383: expected 20 fields, saw 43
Skipping line 476387: expected 20 fields, saw 34
Skipping line 476422: expected 20 fields, saw 21
Skipping line 476427: expected 20 fields, saw 22
Skipping line 476439: expected 20 fields, saw 22
Skipping line 476451: expected 20 fields, saw 56
Skipping line 476463: expected 20 fields, saw 21
Skipping line 476470: expected 20 fields, saw 33
Skipping line 4764

In [ ]:
data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,"""Product arrived labeled as Jumbo Salted Peanu..."
2,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
3,9,B000E7L2R4,A1MZYO9TZK0BBI,R. James,1,1,5,1322006400,Yay Barley,Right now I'm mostly just sprouting this so my...
4,10,B00171APVA,A21BT40VZCCYT4,Carol A. Reed,0,0,5,1351209600,Healthy Dog Food,This is a very healthy dog food. Good for thei...


In [ ]:
print(data["Score"].value_counts())
print(data["Score"].isnull().sum())

5                            193129
5                             46494
4                             40182
1                             27637
3                             20847
2                             15211
4                              7243
1                              5965
3                              3215
2                              2619
0                               847
7                                30
6                                28
book-blogger"""                  20
9                                15
 and Reviewer ""Todd ..."        12
8                                11
10                               11
 and Kitten"""                   10
11                                7
14                                7
25                                6
20                                6
 RN"""                            6
17                                6
 ..."                             5
33                                5
22                          

In [ ]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import string
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

# Load the dataset
df = pd.read_csv("Reviews.csv")

# 1. Split the dataset into a train set and a test set (80% train, 20% test)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# 2. Data Preprocessing
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])
    # Perform stemming using Porter Stemmer
    stemmer = PorterStemmer()
    text = ' '.join([stemmer.stem(word) for word in text.split()])
    return text

train_df['Processed_Text'] = train_df['Text'].apply(preprocess_text)
test_df['Processed_Text'] = test_df['Text'].apply(preprocess_text)

# 3. Feature Extraction: BOW, n-gram (choose n), tf-idf
# TF-IDF with N-gram
#ngram_vectorizer = TfidfVectorizer(ngram_range=(1, 2))  # Adjust ngram_range as needed
#X_train_ngram_tfidf = ngram_tfidf_vectorizer.fit_transform(train_df['Processed_Text'])
#X_test_ngram_tfidf = ngram_tfidf_vectorizer.transform(test_df['Processed_Text'])

# BOW
bow_vectorizer = CountVectorizer()
X_train_bow = bow_vectorizer.fit_transform(train_df['Processed_Text'])
X_test_bow = bow_vectorizer.transform(test_df['Processed_Text'])

# TF-IDF
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(train_df['Processed_Text'])
X_test_tfidf = tfidf_vectorizer.transform(test_df['Processed_Text'])

# 4. Modeling

# Modeling with N-gram TF-IDF
# Using Logistic Regression
#lr_model_ngram = LogisticRegression(max_iter=50)
#lr_model_ngram.fit(X_train_ngram_tfidf, train_df['Score'])
#lr_predictions_ngram = lr_model_ngram.predict(X_test_ngram_tfidf)

# Using Support Vector Machine
#svm_model_ngram = SVC(max_iter=50)
#svm_model_ngram.fit(X_train_ngram_tfidf, train_df['Score'])
#svm_predictions_ngram = svm_model_ngram.predict(X_test_ngram_tfidf)

# Using Logistic Regression
lr_model = LogisticRegression()
lr_model.fit(X_train_bow, train_df['Score'])
lr_predictions_bow = lr_model.predict(X_test_bow)

lr_model.fit(X_train_tfidf, train_df['Score'])
lr_predictions_tfidf = lr_model.predict(X_test_tfidf)

# Using Support Vector Machine
svm_model = SVC()
svm_model.fit(X_train_bow, train_df['Score'])
svm_predictions_bow = svm_model.predict(X_test_bow)

svm_model.fit(X_train_tfidf, train_df['Score'])
svm_predictions_tfidf = svm_model.predict(X_test_tfidf)

# 5. Model Evaluation
def evaluate_model(predictions, true_labels):
    accuracy = accuracy_score(true_labels, predictions)
    confusion_mat = confusion_matrix(true_labels, predictions)
    return accuracy, confusion_mat

# Logistic Regression results
lr_accuracy_bow, lr_confusion_bow = evaluate_model(lr_predictions_bow, test_df['Score'])
lr_accuracy_tfidf, lr_confusion_tfidf = evaluate_model(lr_predictions_tfidf, test_df['Score'])

# Support Vector Machine results
svm_accuracy_bow, svm_confusion_bow = evaluate_model(svm_predictions_bow, test_df['Score'])
svm_accuracy_tfidf, svm_confusion_tfidf = evaluate_model(svm_predictions_tfidf, test_df['Score'])

# 6. Tune the model's hyperparameters (if needed)

# Print results
print("Logistic Regression - BOW Accuracy:", lr_accuracy_bow)
print("Logistic Regression - TF-IDF Accuracy:", lr_accuracy_tfidf)
print("SVM - BOW Accuracy:", svm_accuracy_bow)
print("SVM - TF-IDF Accuracy:", svm_accuracy_tfidf)

print("\nLogistic Regression - BOW Confusion Matrix:")
print(lr_confusion_bow)
print("\nLogistic Regression - TF-IDF Confusion Matrix:")
print(lr_confusion_tfidf)

print("\nSVM - BOW Confusion Matrix:")
print(svm_confusion_bow)
print("\nSVM - TF-IDF Confusion Matrix:")
print(svm_confusion_tfidf)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Logistic Regression - BOW Accuracy: 0.7146752275825674
Logistic Regression - TF-IDF Accuracy: 0.7137517041206737
SVM - BOW Accuracy: 0.4695017371036545
SVM - TF-IDF Accuracy: 0.19464356392101675

Logistic Regression - BOW Confusion Matrix:
[[ 1287   164   110    82   478]
 [  296   245   174   110   341]
 [  157   150   424   310   681]
 [   81    68   182   800  2102]
 [  185    78   147   592 13495]]

Logistic Regression - TF-IDF Confusion Matrix:
[[ 1237   104    90    59   631]
 [  281   175   147    92   471]
 [  160    91   366   283   822]
 [   68    38   163   638  2326]
 [  150    37   102   394 13814]]

SVM - BOW Confusion Matrix:
[[ 854   51   26   40 1150]
 [ 492   25   16   30  603]
 [ 779   35   47   38  823]
 [1291   47   77  108 1710]
 [4129  221  176  329 9642]]

SVM - TF-IDF Confusion Matrix:
[[1405  330  165  175   46]
 [ 597  221  166  158   24]
 [ 735  296  390  260   41]
 [1201  360  655  829  188]
 [5599 1472 1850 3995 1581]]
